<a href="https://colab.research.google.com/github/saipragna25/Special_Topics_LLM_finetuning_assignment/blob/main/task%20F/ST_LLM_Finetuning_f.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing required libraries

In [1]:
!pip install pypdf
!pip -q install accelerate
!pip install llama-cpp-python
!pip -q install git+https://github.com/huggingface/transformers
!pip install langchain
!pip install faiss-cpu
!pip install CTransformers
!pip install huggingface-hub
!pip install torch
!pip install sentence_transformers



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 45.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.18-cp310-cp310-manylinux_2_35_x86_64.whl size=1962442 sha256=afe20f9e874614eb54cab31d815ee7fa214812b1f32d84e6e1bdf54205f64d26
  Stored in directory: /root/.cache/pip/wheels/75/65/c3/54e9fca551b4734cc17f41fafb596a7807312fb470c806e1ab
Successfully built llama-cpp-python
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

Importing Required Libraries

In [2]:
import torch
from langchain.vectorstores import FAISS
from langchain import PromptTemplate, LLMChain
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.llms import LlamaCpp, CTransformers
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings

Now, let's use the Mistral 7B model as backend

In [3]:
model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf'
model_name = "TheBloke/Mistral-7B-Instruct-v0.1-GGUF"
model_file = "mistral-7b-instruct-v0.1.Q4_K_M.gguf"

In [4]:
from google.colab import drive

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
llm = CTransformers(
    model=model_name,
    model_file=model_file,
    temperature=0.1,
    max_new_tokens=512,)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

mistral-7b-instruct-v0.1.Q4_K_M.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

Now, let's demonstrate with some simple prompts

In [8]:
recipe_query = """Provide a brief description of the recipe for {dish_name}. Key Ingredients: {ingredients}. Steps: {steps}"""
ingredients = "Pasta, tomatoes, garlic, olive oil, basil, Parmesan cheese"
steps = "Boil pasta, prepare sauce with tomatoes and garlic, mix with pasta, garnish with basil and Parmesan"
recipe_prompt = PromptTemplate(template=recipe_query, input_variables=["dish_name", "ingredients", "steps"])
llm_chain_recipe = LLMChain(prompt=recipe_prompt, llm=llm)
response_recipe = llm_chain_recipe.run({"dish_name": "Simple Pasta Dish", "ingredients": ingredients, "steps": steps})

In [9]:
response_recipe

' cheese.\n\nRecipe for Simple Pasta Dish:\n\nIngredients:\n- 8 oz pasta (e.g., spaghetti or linguine)\n- 4 medium-sized tomatoes, chopped\n- 3 cloves garlic, minced\n- 2 tablespoons olive oil\n- 1/4 cup fresh basil leaves, chopped\n- 1/2 cup grated Parmesan cheese\n- Salt and pepper to taste\n\nInstructions:\n\n1. Bring a large pot of salted water to a boil. Add the pasta and cook according to package instructions until al dente.\n\n2. Meanwhile, prepare the sauce. In a small saucepan, heat the olive oil over medium heat. Add the minced garlic and sauté for 1-2 minutes until fragrant.\n\n3. Add the chopped tomatoes to the saucepan with the garlic. Season with salt and pepper. Cook for about 5-7 minutes, stirring occasionally, until the tomatoes have released their juices and created a slightly thickened sauce.\n\n4. Once the pasta is cooked, reserve about one cup of the pasta'

In [10]:
plant_caring_query = """Provide detailed care instructions for {plant_name}. Sunlight requirements: {sunlight}. Watering needs: {watering}. Additional care tips: {additional_care}"""
sunlight = "Indirect, moderate sunlight"
watering = "Keep soil consistently moist, avoid overwatering"
additional_care = "High humidity, avoid cold drafts, fertilize monthly during growing season"
plant_care_prompt = PromptTemplate(template=plant_caring_query, input_variables=["plant_name", "sunlight", "watering", "additional_care"])
llm_chain_plant_care = LLMChain(prompt=plant_care_prompt, llm=llm)
response_plant_care = llm_chain_plant_care.run({"plant_name": "Fern", "sunlight": sunlight, "watering": watering, "additional_care": additional_care})

In [11]:
response_plant_care

".\n\nFerns are a popular choice for those looking to add some greenery to their home. These beautiful plants require minimal maintenance but do have specific care requirements that should be followed in order to keep them thriving. Here's what you need to know about how to care for ferns:\n\nSunlight Requirements:\nFerns prefer indirect, moderate sunlight. Direct sun can scorch the leaves and cause the plant to dry out. If your fern is in a brightly lit area, make sure it has adequate protection from the sun or consider moving it to a more shaded location.\n\nWatering Needs:\nFerns need consistently moist soil, but avoid overwatering. Allow the top inch of soil to dry out between waterings, then water thoroughly until you see water draining from the bottom of the pot. Be sure to empty any excess water from saucers or trays to prevent root rot.\n\nAdditional Care Tips:\nFerns thrive in high humidity environments, so avoid cold drafts and keep the area around your fern warm and moist. I